## price_tag_detection and crop _ 230105

[Roboflow : How to Use the price tag detection Detection API](https://universe.roboflow.com/sovar/price-tag-detection-r5jlv/model/1)

- price_tag을 인식한 후 price_tag으로 인식된 box 부분만 crop하여 저장하는 코드입니다.

- 원본 파일 59개 중 39개의 파일에서 price_tag가 인식되었습니다.
 

In [ ]:
from roboflow import Roboflow
from PIL import Image
import supervision as sv
import cv2
import os

#from google.colab import drive
#drive.mount('/content/drive')
%matplotlib inline

rf = Roboflow(api_key="")
project = rf.workspace().project("price-tag-detection-r5jlv")
model = project.version(1).model

def price_tag(image_path, i):
    image = cv2.imread(image_path)
    result = model.predict(image, confidence=40, overlap=30).json()
    print('result: ',result)
    labels = [item["class"] for item in result["predictions"]]
    detections = sv.Detections.from_roboflow(result)
    label_annotator = sv.LabelAnnotator()
    mask_annotator = sv.MaskAnnotator()

    annotated_image = mask_annotator.annotate(
        scene=image, detections=detections)
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections, labels=labels)

    output_image_path = f'image_path.png'
    #cv2.imwrite(output_image_path, annotated_image)
    if result["predictions"]:
      x = int(result["predictions"][0]["x"])
      y = int(result["predictions"][0]["y"])
      width = int(result["predictions"][0]["width"])
      height = int(result["predictions"][0]["height"])
      area = (x-(width/2),y - (height/2),x+(width/2),y+(height/2))
      to_crop = Image.open(image_path)
      tc = to_crop.copy()
      crop_image = tc.crop(area)
      crop_image.save(f'save_image_path{i}.jpg')
    sv.plot_image(image=annotated_image, size=(8, 8))

    return output_image_path

path = '/content/drive/MyDrive/ocr_data' # 원본 데이터
file_list = os.listdir(path)
file_list_py = [file for file in file_list]
i = 0
for f in file_list_py:
    image_path = os.path.join(path, f)
    saved_image_path = price_tag(image_path, i)
    print(f"Image saved to: {saved_image_path}")
    i += 1